___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Регуляризация в SciKit-Learn

Ранее мы создали набор полиномиальных признаков и затем применили для них линейную регрессию. Но мы можем сделать модель ещё лучше, используя регуляризацию.

Регуляризация старается минимизировать метрику "сумма квадратов остатков" (RSS - residual sum of squares) *и* "функцию штрафа". Функция штрафа старается "наказывать" модели с очень большими коэффициентами. Некоторые методы регуляризации по сути приводят к тому, что некоторые неполезные признаки получают нулевые коэффициенты, то есть модель не будет использовать эти признаки.

Давайте рассмотрим два метода регуляризации - гребневая регрессия (или ридж-регрессия, или регрессия риджа, англ. ridge regression) и регрессия лассо. Мы будем использовать их в сочетании с полиномиальными признаками, поскольку регуляризация модели не так эффективна на исходном наборе признаков набора данных X.

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Загрузка данных и подготовительные шаги

In [2]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

### Полиномиальная конвертация

In [3]:
from sklearn.preprocessing import PolynomialFeatures

In [4]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [5]:
poly_features = polynomial_converter.fit_transform(X)

### Разбиение на обучающий и тестовый наборы данных -  Train | Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

----
----

## Масштабирование данных

Хотя в наших данных все признаки имеют один и тот же порядок величины (в $1000 долларов), это не всегда верно в других наборах данных. Поскольку математика регуляризации будет суммировать коэффициенты, очень важно стандартизировать признаки. Посмотрите теоретические видео-лекции для более подробного рассказа на эту тему, а также почему мы делаем **обучение (fit)** только на обучающих данных, и делаем **трансформацию** обоих наборов данных отдельно друг от друга.

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# help(StandardScaler)

In [10]:
scaler = StandardScaler()

In [11]:
scaler.fit(X_train)

StandardScaler()

In [12]:
X_train = scaler.transform(X_train)

In [13]:
X_test = scaler.transform(X_test)

## Гребневая регрессия (Ridge Regression)

Посмотрите видео-лекции по этой теме - там мы рассказываем про гребневую регрессию и про выбор параметра alpha.

In [14]:
from sklearn.linear_model import Ridge

In [15]:
ridge_model = Ridge(alpha=10)

In [16]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [17]:
test_predictions = ridge_model.predict(X_test)

In [18]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [19]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [20]:
MAE

0.5774404204714174

In [21]:
RMSE

0.894638646131967

Какова точность на обучающей выборке? (Мы будем использовать это позже для сравнения)

In [22]:
# Производительность на обучающем наборе
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.528834818302532

### Выбираем значение alpha с помощью кросс-валидации

Детали есть в видео-лекции.

In [23]:
from sklearn.linear_model import RidgeCV

In [24]:
# help(RidgeCV)

In [25]:
# Выбираем способ оценки модели: https://scikit-learn.org/stable/modules/model_evaluation.html
# Отрицательные значения RMSE. Все метрики следуют принципу "больше значит лучше".

# Смотрим все варианты: sklearn.metrics.SCORERS.keys()
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [26]:
# Чем больше значений alpha Вы укажете, тем дольше времени потребуется.
# К счастью наш набор данных достаточно маленький.
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [27]:
ridge_cv_model.alpha_

0.1

In [28]:
test_predictions = ridge_cv_model.predict(X_test)

In [29]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [30]:
MAE

0.42737748843333817

In [31]:
RMSE

0.6180719926938041

In [32]:
# Метрика MAE на обучающем наборе
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.30941321056342724

In [33]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])


-----

## Регрессия Лассо

In [34]:
from sklearn.linear_model import LassoCV

In [35]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [36]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [37]:
lasso_cv_model.alpha_

0.49430709092258285

In [38]:
test_predictions = lasso_cv_model.predict(X_test)

In [39]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [40]:
MAE

0.6541723161252858

In [41]:
RMSE

1.1308001022762542

In [42]:
# Метрика MAE на обучающем наборе
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.69128071408207

In [43]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Elastic Net

Elastic Net сочетает в себе ридж-регрессию и лассо-регрессию, чтобы взять лучшее из обеих регуляризаций!

In [44]:
from sklearn.linear_model import ElasticNetCV

In [45]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [46]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [47]:
elastic_model.l1_ratio_

1.0

In [48]:
test_predictions = elastic_model.predict(X_test)

In [49]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [50]:
MAE

0.5663262117569451

In [51]:
RMSE

0.7485546215633726

In [52]:
# Метрика MAE на обучающем наборе
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.4307582990472368

In [53]:
elastic_model.coef_

array([ 3.78993643,  0.89232919,  0.28765395, -1.01843566,  2.15516144,
       -0.3567547 , -0.271502  ,  0.09741081,  0.        , -1.05563151,
        0.2362506 ,  0.07980911,  1.26170778,  0.01464706,  0.00462336,
       -0.39986069,  0.        ,  0.        , -0.05343757])

-----
---